In [ ]:
# 0. Setup the libraries and input data

# 0.2 Load libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statistics #staticstics calculates variance the same way as var in R


In [ ]:
# 0.3 Load the input data
LogRatios = pd.read_csv('Homework6_log2ratios_chr1.txt', sep='\t')
x = LogRatios['log2Ratio'].values
copyNumberStates = np.array([1,2,3,4,5])
K = len(copyNumberStates)

In [ ]:
# 0.4. Initialize model parameters and hyperparameters.

pi_init = np.array([0.1,0.6,0.1,0.1,0.1]) 
mu_init = np.log2(copyNumberStates/2) 
var_init = np.array([statistics.variance(x) for m in range(K)]) 

A_init = np.zeros([K,K]) 
for i in range(K):
    A_init[i,] = (1- 0.99999)/(K-1)
    A_init[i,i] = 0.99999
    
deltaPi_hyper = [2,6,2,2,2] 

mMu_hyper = mu_init
sMu_hyper = var_init

betaVar_hyper = np.array([1,1,1,1,1])
alphaVar_hyper = betaVar_hyper/(statistics.variance(x)/np.sqrt(K))

dirCounts = 100000
deltaA_hyper = A_init * dirCounts

In [ ]:
# 1. Compute the Gaussian Emission Probabilities

# use scipy.stats.norm for the normal pdf
from scipy.stats import norm
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

# TO DO: 1.1. Define a function to compute the likelihood probabilities:

def compute_gauss_lik():
    """
    write code here
    """


In [ ]:
# 1.2. TO DO: Compute the Gaussian likelihood
"""
use function from 1.1
"""


In [ ]:
#this is code for the foward backward algorithm to use instead of the R package HMMCopy forward_backward

def forward_backward(pi, A, obs_lik):
    
    #initialize
    Q,T = obs_lik.shape 
    scale = np.ones(T)
    alpha = np.zeros([Q,T])

    alpha[:,0] = obs_lik[:,0]*pi 

    scale[0] = sum(alpha[:,0])
    alpha[:,0]=alpha[:,0]/scale[0]

    #forwards
    for t in range(1,len(obs_lik.T)):
        m = np.matmul(A.T,alpha[:,t-1])

        alpha[:,t] = m * obs_lik[:,t]

        scale[t] = sum(alpha[:,t]) 
        alpha[:,t]=alpha[:,t]/scale[t]

    loglik = sum(np.log(scale))

    #backwards
    beta = np.zeros([Q,T])
    beta[:,T-1] = np.ones(Q)

    rho = np.zeros([Q,T])
    rho[:,T-1] = alpha[:,T-1]

    xi   = np.zeros([Q,Q,T])

    for t in reversed(range(0,T-1)):
        b = beta[:,t+1] * obs_lik[:,t+1]

        beta[:,t]=np.matmul(A,b)
        beta[:,t]=beta[:,t]/sum(beta[:,t])

        rho[:,t] = alpha[:,t] * beta[:,t]
        rho[:,t]=rho[:,t]/sum(rho[:,t])

        xi[:,:,t]= A * np.matmul(alpha[:,t].reshape(-1,1), b.reshape(-1,1).T) 

        xi[:,:,t] = xi[:,:,t]/sum(sum(xi[:,:,t]))

    return (rho,xi,loglik)

In [ ]:
# 2.1. TO DO: Compute the responsibilities in the E-Step

#example syntax for using the forward_backward function
rho,xi,loglik = forward_backward(pi, A, obs_lik)

"""
write code here
"""

In [ ]:
# 2.2. Updating the initial state distribution parameter pi1:K in the M-Step 
    
# TO DO: 2.2.1. Write a function to update the initial state distribution parameter

def update_pi():
    """
    write code here
    """

In [ ]:
# 2.2.2. Compute pi_hat for the first iteration of EM
"""
use function from 2.2.1
"""

In [ ]:
# 2.3. Updating the Gaussian mean parameter mu_1:K in the M-Step 

# TO DO: 2.3.1. Write a function to update Gaussian mean parameter

def update_mu():
    """
    write code here
    """

In [ ]:
# 2.3.2. Compute mu_hat for the first iteration of EM
"""
use function from 2.3.1
"""

In [ ]:
# 2.4. Updating the Gaussian variance parameter in the M-Step

# 2.4.1. TO DO: Write a function to update Gaussian variance parameter

def update_var():
    """
    write code here
    """

In [ ]:
# 2.4.2. Compute the updated variance (var_hat) for the first iteration of EM
"""
use function from 2.4.1
"""

In [ ]:
# 2.5. Updating the transition probabilities A in the M-Step 

# TO DO : 2.5.1. Write a function to update the transition probabilities

def update_A():
    """
    write code here
    """

In [ ]:
# 2.5.2. Compute A_hat for the first iteration of EM
"""
use function from 2.5.1
"""

In [ ]:
# 2.6. Compute the log posterior
# 2.6.1. Write a function, compute_log_posterior, to compute the log posterior distribution.

def compute_log_posterior ():
    """
    write code here
    """

In [ ]:
# 2.6.2. Compute the log posterior for the input data
"""
use function from 2.6.1
"""

In [ ]:
# 3. Learn the HMM Parameters and Predict the Copy Number Segments.

"""
write code here
"""

"""
After convergence:

i. Print out converged parameters:
    the Gaussian mean and variance parameters mu_hat1:K and var_hat1:K
    the initial state distribution pi_hat1:K
    the transition matrix A_hat
ii. Print out the log posterior for all iterations.
iii. Save the Gaussian likelihood probabilities obs.lik computed in the final iteration. You will need this for the Viterbi algorithm in Section 3.2.
"""



In [ ]:
#this is code for the viterbi algorithm to use instead of the R package HMMCopy Viterbi

def viterbi(log_pi, log_A, log_obs_lik):
    Q,T = log_obs_lik.shape
    
    delta = np.zeros([Q,T])
    psi = np.zeros([Q,T])
    path = np.zeros(T)

    delta[:,0] = log_pi + log_obs_lik[:,0] 
    
    psi[:,0] = 0
    
    #forward
    for t in range(1,T): 
        for j in range(0,Q):
            
            delta[j,t] = np.max(delta[:,t-1] + log_A[:,j])
            
            psi[j,t] = np.argmax(delta[:,t-1] + log_A[:,j])
            
            delta[j,t] = delta[j,t] + log_obs_lik[j,t]

    path = delta.argmax(axis=0)

    #backwards
    for t in reversed(range(0,T-1)): 
        path[t] = psi[path[t+1],t+1]         

    return(path)


In [ ]:
# 3.2. Determine the copy number segments using the Viterbi algorithm
path = viterbi(np.log(pi[-1]),np.log(A[-1]),np.log(obs_lik))

#TO DO: print out the segments

"""
write code here
"""


In [ ]:
# TO DO: 4.1. Implement the function to calculate power.

def computePower():
    """
    write code here
    """


In [ ]:
# 4.2. What is the power for depth N=40 and tumor fraction alpha = 0.2
"""
write code here
"""

In [ ]:
# TO DO: 4.3. Plot a graph of the power estimates (y-axis) for a range of tumor fractions (x-axis)

"""
write code here
"""


In [ ]:
# TO DO: 4.4. Plot a graph of the power (y-axis) required for a range of read depths (x-axis)

"""
write code here
"""


In [ ]:
# 4.5. What is the minimum depth required for tumor fraction alpha = 0.1 and desired power >= 0.8?

"""
write code here
"""
